In [1]:
#  相關套件

import time
import random
import json
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import pandas as pd
from collections import deque
from concurrent.futures import ThreadPoolExecutor

WEB_NAME = 'Cake_me'

In [2]:
# 取得職業類別

def category_func(prefix):
    job_url = "https://www.cake.me/job"
    jobtt = requests.get(job_url)
    jobtt_soup = BeautifulSoup(jobtt.text, 'html.parser')
    scripts = jobtt_soup.find_all('script')
    df = pd.json_normalize(json.loads(scripts[-1].string)['props']['pageProps']['_nextI18Next'])
    filtered_columns = [col for col in df.columns if col.startswith(prefix)]
    new_column_names = [col.replace(prefix, '') for col in filtered_columns]
    filtered_df = df[filtered_columns].copy()
    filtered_df.columns = new_column_names
    return filtered_df

# 職業類別細項 sector
# sector_prefix = 'initialI18nStore.zh-TW.sector.sectors.'
# df_sector = category_func(sector_prefix)
# df_sector

# 職業類別  sector_groups
sector_groups_prefix = 'initialI18nStore.zh-TW.sector.sector_groups.'
df_sector_groups = category_func(sector_groups_prefix)
df_sector_groups

,advertising-marketing-agency,agriculture,architecture,banking-insurance-finance,consulting-audit,corporate-services,culture-media-entertainment,design-art,distribution,education-training-recruitment,...,hotel-tourism-leisure,industry,legal-law,medical,mobility-transport,non-profit-association,public-administration,real-estate,service-industry,tech
0,廣告 / 行銷 / 代理,農林漁牧業,建築設計,銀行 / 保險 / 金融,顧問 / 審計,公司服務,文化 / 媒體 / 娛樂,設計 / 藝術,分銷,教育 / 培訓 / 招聘,...,飯店 / 旅遊 / 休閒,工業,法律 / 法規,生醫 / 醫療,移動 / 運輸,非營利 / 社團組織,公共行政,房地產,服務,科技


In [3]:
# 產生cake.me網址 https://www.cake.me 根據提供的(關鍵字和職缺類別) 轉換為職缺網址

def cake_me_url(KEYWORDS, CATEGORY, ORDER=None):
    """
    這個函數會根據給定的關鍵字和類別參數構建一個完整的職缺網址。
    如果同時提供了關鍵字和類別，將會包含兩者；如果只提供其中一個，則只會包含該參數。

    參數:
    KEYWORDS (str): 職缺的關鍵字。
    CATEGORY (str): 職缺的類別。
    ORDER (str, optional): 排序的參數，預設為 None。

    返回:
    str: 生成的職缺網址。

    # 測試範例
    url_1 = cake_me_url("雲端工程師", "it", "latest")    
    # https://www.cake.me/jobs/雲端工程師?order=latest&profession[0]=it&page=

    url_2 = cake_me_url("雲端工程師", "")      
    # https://www.cake.me/jobs/雲端工程師?page=

    url_3 = cake_me_url("", "it", "latest")             
    # https://www.cake.me/jobs/categories/it?order=latest&page=

    url_4 = cake_me_url("", "")               
    # https://www.cake.me/jobs?page=
    
    """

    BASE_URL = "https://www.cake.me/jobs"

    if KEYWORDS and CATEGORY:
        url = f"{BASE_URL}/{KEYWORDS}?profession[0]={CATEGORY}&page="
    elif KEYWORDS:
        url = f"{BASE_URL}/{KEYWORDS}?page="
    elif CATEGORY:
        url = f"{BASE_URL}/categories/{CATEGORY}?page="
    else:
        url = f"{BASE_URL}?page="

    if ORDER:  # 只在 ORDER 不為 None 時添加
        url = url.replace("?page=", f"?order={ORDER}&page=")

    return url


# # 測試範例  類別: {軟體, it}
# url_1 = cake_me_url("雲端工程師", "it", "latest")    
# print(url_1)  # https://www.cake.me/jobs/雲端工程師?order=latest&profession[0]=it&page=

# url_2 = cake_me_url("雲端工程師", "")      
# print(url_2)  # https://www.cake.me/jobs/雲端工程師?page=

# url_3 = cake_me_url("", "it", "latest")             
# print(url_3)  # https://www.cake.me/jobs/categories/it?order=latest&page=

# url_4 = cake_me_url("", "")               
# print(url_4)  # https://www.cake.me/jobs?page=

In [4]:
#  從指定的職缺網址獲取工作職缺的網址

def fetch_job_url(joburl):
    """
    這個函數會遍歷多個頁面，並從每個頁面中提取工作職缺的網址，將其存儲在一個集合中以避免重複。
    使用 tqdm 顯示進度條，並在每次請求之間隨機延遲以避免過於頻繁的請求。

    參數:
    joburl (str): 職缺列表的基本網址。
    PAGE (int): 起始頁碼。

    返回:
    list: 包含所有獲取到的工作職缺網址的列表。
    """

    PAGE = 0
    MAX_PAGE = 1

    MAX_LENGTH = 4
    recent_counts = deque(maxlen=MAX_LENGTH)

    job_url_set = set()  # 使用 set() 來存儲網址
    with tqdm(total=MAX_PAGE, desc="cake.me職缺列表 ", unit="PAGE", leave=True) as pbar:
        while True:
            # 獲取當前頁面內容的工作網址
            response = requests.get(f"{joburl}{PAGE}")
            response_soup = BeautifulSoup(response.text, 'html.parser')
            job_urls = response_soup.find_all('a', class_='JobSearchItem_jobTitle__bu6yO')
            for job_url in job_urls:
                job_url_set.add("https://www.cake.me" + job_url['href'])  # 添加到 set 中
            
            # 檢查是否有新資料
            total_jobs = len(job_url_set) 
            recent_counts.append(total_jobs)
            if len(recent_counts) == MAX_LENGTH and len(set(recent_counts)) == 1:
                print(f"連續{MAX_LENGTH}次沒有新資料，提前結束。")
                print(f"Total unique job URLs fetched: {len(job_url_set)}")
                break
               
            # 獲取總頁數
            time.sleep(random.uniform(0.5, 1.5))
            pagination_items = response_soup.find_all('a', class_='Pagination_itemNumber___enNq')
            if pagination_items:
                MAX_PAGE = int(pagination_items[-1].text) + 1
                pbar.total = MAX_PAGE  # 更新進度條的總頁數
            pbar.set_postfix_str(f"目前頁面 {PAGE}, 最大頁數: {MAX_PAGE}")
            pbar.update(1)

            if PAGE <= MAX_PAGE:  
                PAGE = PAGE + 1 
            else:
                break

        return list(job_url_set)  # 將 set 轉換為 list
    
    
# 測試範例
# joburl = "https://www.cake.me/jobs/雲端工程師?order=latest&profession[0]=it&page="
# job_urls = fetch_job_url(joburl)
# job_urls[0]

In [5]:
# 從指定的職缺網址獲取職缺的相關數據

def fetch_job_data(job_url):
    """
    這個函數會發送 GET 請求到提供的職缺網址，並使用 BeautifulSoup 解析返回的 HTML 文檔。
    它會從頁面中的 JavaScript 代碼中提取職缺的元數據，並將其轉換為 Pandas DataFrame 格式。

    參數:
    job_url (str): 職缺的網址。

    返回:
    pd.DataFrame: 包含職缺詳細信息的 DataFrame，包括職缺網址、公司名稱、公司網址及其他職缺網址。
    """
    
    response = requests.get(job_url)
    response_soup = BeautifulSoup(response.text, 'html.parser')
    scripts = response_soup.find_all('script')
    
    # if not scripts or not scripts[-1].string:
    #     print(f"No script tags found or last script is empty for URL: {job_url}")
    #     return None
    
    # try:
    #     jobMetaData = json.loads(scripts[-1].string)['props']['pageProps']['ssr']['jobMetaData']['job']
    # except json.JSONDecodeError as e:
    #     print(f"JSONDecodeError: {e} - URL: {job_url} - Content: {scripts[-1].string}")
    #     return None
    
    jobMetaData = json.loads(scripts[-1].string)['props']['pageProps']['ssr']['jobMetaData']['job']

    df = pd.json_normalize(jobMetaData)

    # 公司名稱、網址、其他職缺、目前職缺網址
    office_name = json.loads(scripts[1].string)['itemListElement'][0]['item']['name']
    office_url = json.loads(scripts[1].string)['itemListElement'][0]['item']['@id']
    job_other = json.loads(scripts[1].string)['itemListElement'][1]['item']['@id']
    job_url = json.loads(scripts[1].string)['itemListElement'][2]['item']['@id']
    df['job_url'] = job_url
    df['office_name'] = office_name
    df['office_url'] = office_url
    df['job_other'] = job_other
    
    return df

# # 測試範例
# job_url = 'https://www.cake.me/companies/qdm/jobs/2f3db0'  # job_urls[0]
# job_data = fetch_job_data(job_url)
# job_data

In [6]:
# 根據關鍵字與職業類別 獲取所有工作職位的資料

SEARCH_TIMESTAMP = time.strftime('%Y-%m-%d', time.localtime(time.time()))
KEYWORDS = "雲端工程師"
CATAGORY = "it"
FILE_NAME = f"({SEARCH_TIMESTAMP})_{WEB_NAME}_{KEYWORDS}_{CATAGORY}"

print( f"開始執行 {FILE_NAME}" )
job_data_list = []
SEARCH_PAGE_URL = cake_me_url(KEYWORDS, CATAGORY)       # 產生cake.me網址
job_urls = fetch_job_url(SEARCH_PAGE_URL)               # 列出職缺列表
with ThreadPoolExecutor(max_workers=30) as executor:    # 列出職缺細項
    futures = [executor.submit(fetch_job_data, url) for url in job_urls]
    for future in tqdm(futures):
        result = future.result()
        if result is not None and not result.empty:  # 確保結果不為空
            job_data_list.append(result)

all_jobs_df = pd.concat(job_data_list, axis=0)  
all_jobs_df.reset_index(drop=True, inplace=True)
all_jobs_df.shape

開始執行 (2025-05-23)_Cake_me_雲端工程師_it


cake.me職缺列表 : 100%|██████████| 32/32 [01:43<00:00,  3.23s/PAGE, 目前頁面 31, 最大頁數: 32]


連續4次沒有新資料，提前結束。
Total unique job URLs fetched: 282


100%|██████████| 282/282 [00:21<00:00, 12.97it/s]


(282, 58)

In [7]:
# 使用 rename 方法 將 all_jobs_df.columns 改為中文

RENAME_DICT = {
    'id': '職缺ID',
    'title': '職缺標題',
    'description': '職缺描述tag',
    'path': '職缺路徑代碼',
    'requirements': '職務需求',
    'interview_process': '面試流程',
    'salary_type': '薪資類型',
    'salary_min': '最低薪資',
    'salary_max': '最高薪資',
    'hide_salary_completely': '完全隱藏薪資',
    'hide_salary_max': '隱藏最高薪資',
    'number_of_openings': '職缺人數',
    'position': '職位',
    'content_updated_at': '內容更新時間',
    'salary_currency': '薪資幣別',
    'job_type': '職缺型態',
    'seniority_level': '職級類型',
    'category': '職業類別',
    'sponsored': '贊助',
    'expired_at': '到期時間',
    'expired_period': '到期期間',
    'signing_bonus': '簽約獎金',
    'tag_list': '技能需求列表',
    'location_list': '地點列表',
    'google_places': 'Google地點',
    'cr_locations': 'CR地點',
    'aasm_state': 'AASM狀態',
    'suspended': '暫停',
    'use_external_url': '使用外部網址',
    'external_url': '外部網址',
    'number_of_management': '管理人數',
    'year_of_seniority': '年資',
    'min_work_exp_year': '最低工作經驗年數',
    'description_plain_text': '職缺描述',
    'created_at': '創建時間',
    'job_or_page_currency': '頁面薪資幣別',
    'application_read_rate': '應徵者點閱率',
    'candidate_read_rate': '頁面點閱率',
    'candidate_read_time': '頁面停留時間',
    'inclusivity_traits': '包容性特徵',
    'profession': '職業細項',
    'page_path': '頁面路徑',
    'page_name': '頁面名稱',
    'page_country': '頁面國家',
    'remote.key': '遠端工作性質',
    'remote.text': '遠端工作文本',
    'remote.can_remote': '可遠端工作',
    'page_logo.url': '頁面標誌網址',
    'page_logo.tiny.url': '頁面標誌小圖網址',
    'page_logo.thumb.url': '頁面標誌縮略圖網址',
    'page_logo.medium.url': '頁面標誌中圖網址',
    'page_logo.large.url': '頁面標誌大圖網址',
    'page_logo.og_image.url': '頁面標誌OG圖像網址',
    'metadata.images': '元數據圖片',
    'job_url': '職缺網址',
    'office_name': '公司名稱',
    'office_url': '公司網址',
    'job_other': '其他職缺'
}


# 存檔
all_jobs_df = all_jobs_df.rename(columns=RENAME_DICT)
all_jobs_df.to_csv(f"{FILE_NAME}.csv", encoding='utf-8-sig')
print(f"存檔完成 : {FILE_NAME}.csv")

all_jobs_df.columns

存檔完成 : (2025-05-23)_Cake_me_雲端工程師_it.csv


Index(['職缺ID', '職缺標題', '職缺描述tag', '職缺路徑代碼', '職務需求', '面試流程', '薪資類型', '最低薪資',
       '最高薪資', '完全隱藏薪資', '隱藏最高薪資', '職缺人數', '職位', '內容更新時間', '薪資幣別', '職缺型態',
       '職級類型', '職業類別', '贊助', '到期時間', '到期期間', '簽約獎金', '技能需求列表', '地點列表',
       'Google地點', 'CR地點', 'AASM狀態', '暫停', '使用外部網址', '外部網址', '管理人數', '年資',
       '最低工作經驗年數', '職缺描述', '創建時間', '頁面薪資幣別', '應徵者點閱率', '頁面點閱率', '頁面停留時間',
       '包容性特徵', '職業細項', '頁面路徑', '頁面名稱', '頁面國家', '遠端工作性質', '遠端工作文本', '可遠端工作',
       '頁面標誌網址', '頁面標誌小圖網址', '頁面標誌縮略圖網址', '頁面標誌中圖網址', '頁面標誌大圖網址', '頁面標誌OG圖像網址',
       '元數據圖片', '職缺網址', '公司名稱', '公司網址', '其他職缺'],
      dtype='object')

In [8]:
# 補充 :  網頁結構解析

url = "https://www.cake.me/jobs/雲端工程師"

response = requests.get(url)
response_soup = BeautifulSoup(response.text, 'html.parser')
scripts = response_soup.find_all('script')


# 查看篩選欄位選項
# options = []
# catagorys = 1
# DropdownButton_contents = response_soup.find_all('div', 'JobSearchPage_searchFilter__ts_A0')
# for JobSearchPage_searchFilter in DropdownButton_contents:
#     title  = JobSearchPage_searchFilter.find('div', 'DropdownButton_content__XZwFf').text
#     # print(title)
#     Checkbox_texts = JobSearchPage_searchFilter.find_all('div', 'Checkbox_text__g6TLq')
#     for Checkbox in Checkbox_texts:
#         # print(Checkbox.text)
#         option_text = Checkbox.text.strip()
#         options.append({'catagorys':catagorys, 'title': title, 'option_text': option_text})
#     catagorys = catagorys +1
# df_searchFilter = pd.DataFrame(options)
# df_searchFilter
# df_searchFilter['option_text'][(df_searchFilter['catagorys']==1)]



# 提取資料結構 使用遞歸函數提取子結構
def extract_keys_with_branch_structure(value_in, current_path=''):
    
    branch_structure = {}
    if isinstance(value_in, dict):
        for index, (key, value) in enumerate(value_in.items()):
            path = f"{current_path}-{index + 1}" if current_path else f"{index + 1}"
            branch_structure[path] = key  # 添加當前層級的鍵

            # path = f"{current_path}-{key}" if current_path else f"{key}"
            # branch_structure[path] = value  # 添加當前層級的鍵
            
            # 使用遞歸查找 子結構、提取與合併
            if isinstance(value, dict):
                branch_structure.update(extract_keys_with_branch_structure(value, path))

    return branch_structure

data = json.loads(scripts[-1].string)
branch_structure = extract_keys_with_branch_structure(data)

json.loads(scripts[-1].string)['props']['pageProps']['serverState']['initialResults']['Job']['state']['query']


'雲端工程師'